<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex04/patrick_ferreira/ex04_175480_patrick_ferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')



Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída). 
- Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

## Importação dos pacotes

In [2]:
import collections
import itertools
import functools
import math
import os
import random
import re

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook, tqdm
from typing import List


from operator import itemgetter

In [3]:
# Check which GPU we are using
!nvidia-smi

Wed Sep 21 22:21:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0    N/A /  N/A |    917MiB /  2004MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


# Carregamento do dataset 

Primeiro, fazemos download do dataset:

In [5]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [6]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
random.shuffle(x_train)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
x_train = x_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x in x_train[:3]:
    print(x[:100])

print('3 últimas amostras treino:')
for x in x_train[-3:]:
    print(x[:100])

print('3 primeiras amostras validação:')
for x in x_valid[:3]:
    print(x[:100])

print('3 últimas amostras validação:')
for x in x_valid[-3:]:
    print(x[:100])



random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
As a collector of movie memorabilia, I had to buy the movie poster for this film which, now that I'v
I've no idea what dimwit from San Francisco came up with this stupid plot, but apparently they need 
I saw the trailer for this film a few months prior to its release, and MAN, did it look scary, Espec
3 últimas amostras treino:
I thought the movie was OK but very disappointed that they didn't capture the true image of his life
I think you would have to be from the USA to get a lot of the jokes. But if you liked Princess Bride
First of all, this film can be divided into three segments. A promising opening, with the ambushing 
3 primeiras amostras validação:
Sorry Fulci fans, but I could not get through this one. The soundtrack was about as annoying as they
I originally saw this movie as a boy at the old Rialto Theatre as part of a Saturday afternoon matin
This film is regard

### Definindo funções de manipulação de texto.

In [7]:
from typing import List

import re
import string


def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """

    return re.findall(r"[\w']+|[.,!?;]", text.lower())


from collections import Counter


def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """

    tokens = []

    for t in texts:
        tokens.extend(tokenize(t))

    return dict(Counter(tokens).most_common(max_tokens))

def items_em_comum(amostra, corpus):
    c = [value for value in amostra if value in corpus]
    return c


def concatenate_list_of_str(texts: List[str]):
    return ".".join(texts)

class Tokenizador():
    def __init__(self, texts, tokenizer=tokenize, vocab_size=3000):
        super().__init__()

        self.tokenizer = tokenizer
        self.vocab_size = vocab_size

        self.every_text = concatenate_list_of_str(texts)
        self.tokens_ocurrences = create_vocab(tokenize(self.every_text), max_tokens=vocab_size)

        self.lista_do_vocabulario = list(self.tokens_ocurrences.keys())

        self.dicionario_tokens = dict(zip(self.lista_do_vocabulario, range(len(self.lista_do_vocabulario))))

    def __call__(self, text):
        tokens = self.tokenizer(text)

        # Pegamos somente os tokens que existem no vocabulario
        tokens_keys = list(items_em_comum(tokens, self.lista_do_vocabulario))

        return list(itemgetter(*tokens_keys)(self.dicionario_tokens))

    def decode(self, ids):
        reversed_dict = dict(zip(self.dicionario_tokens.values(), self.dicionario_tokens.keys()))
        return list(itemgetter(*ids)(reversed_dict))


In [8]:
### Criando classe do dataset

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int, vocab_size=1000):
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            print("Carregando dataset preprocessado")
        except Exception as e:
            # print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenizer(text)
                for i in range(len(tokens_key)-context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+context_size])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long()

Testando Dataset

In [9]:
vocab_size = 3000

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=Tokenizador(dummy_texts, tokenize, vocab_size), context_size=3, vocab_size=vocab_size)
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False)

assert len(dummy_dataset) == 4
print('Passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[1, 2, 0],
     [5, 6, 7]])

correct_first_batch_target = torch.LongTensor([3,   0, ])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

100%|██████████| 2/2 [00:00<00:00, 4412.73it/s]

Montando dataset
Passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


Dados de treino, validação e teste

In [10]:
# Load datasets
context_size = 15

tokenizer = Tokenizador(x_train, tokenize, vocab_size=3000)

training_dataset = MyDataset(texts=x_train, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=x_valid, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=x_test, tokenizer=tokenizer, context_size=context_size)

print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

  0%|          | 31/20000 [00:00<01:05, 306.56it/s]

Montando dataset


  1%|          | 28/5000 [00:00<00:18, 272.48it/s]

Montando dataset


  0%|          | 28/25000 [00:00<01:32, 269.09it/s]

Montando dataset


100%|██████████| 25000/25000 [01:25<00:00, 292.89it/s]


training examples: 4323766
valid examples: 1077234
test examples: 5270830


In [11]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        
        super().__init__()

        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)

        self.features = torch.nn.Sequential(
            nn.Linear(context_size * embedding_dim, hidden_size),
            nn.PReLU(num_parameters=hidden_size,),
            nn.Linear(hidden_size, hidden_size),
            nn.PReLU(num_parameters=hidden_size),
        )

        # self.softmax = nn.Softmax(dim=-1)

        self.classifier = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """

        return self.classifier(
            self.features(
                self.embedding(inputs).view(inputs.shape[0], -1)
            )
        )

Testando se o modelo processa os dados corretamente

In [12]:
model = LanguageModel(
    vocab_size=vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 3000])

Verificando a perplexidade

In [13]:
def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    # Escreva seu código aqui.

    crossentropy =  nn.functional.cross_entropy(logits,target)

    return crossentropy.exp()


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {vocab_size}')

assert math.isclose(my_perplexity, vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              2982
correct initial perplexity: 3000
Passou o no assert da perplexidade


TREINAMENTO

In [14]:
from queue import Queue
from threading import Thread


class DataManager(Thread):
    def __init__(self, data_loader, buffer_size=3, device=torch.device("cpu"), data_type=torch.float32):
        """
This manager intends to load a PyTorch dataloader like from disk into memory,
reducing the acess time. It does not easily overflow memory, because we set a
buffer size limiting how many samples will be loaded at once. Everytime a sample
is consumed by the calling thread, another one is replaced in the
buffer (unless we reach the end of dataloader).

A manger may be called exactly like a dataloader, an it's based in an internal
thread that loads samples into memory in parallel. This is specially useful
when you are training in GPU and processor is almost idle.

        :param data_loader: Base dataloader to load in parallel.
        :param buffer_size: How many samples to keep loaded (caution to not overflow RAM). Default: 3.
        :param device: Torch device to put samples in, like torch.device("cpu") (default). It saves time by transfering in parallel.
        :param data_type: Automatically casts tensor type. Default: torch.float32.
        """
        super().__init__()
        self.buffer_queue = Queue(maxsize=buffer_size)
        self.data_loader = data_loader
        self.buffer_size = buffer_size
        self.device = device
        self.data_type = data_type

        self.dataloader_finished = False

    def run(self):
        """
Runs the internal thread that iterates over the dataloader until fulfilling the
buffer or the end of samples.
        """
        for i, (x, y) in enumerate(self.data_loader):
            # Important to set before put in queue to avoid race condition
            # would happen if trying to get() in next() method before setting this flag
            if i >= len(self) - 1:
                self.dataloader_finished = True

            self.buffer_queue.put([x.to(self.data_type).to(self.device),
                                   y.to(self.data_type).to(self.device)])

    def __iter__(self):
        """
Returns an iterable of itself.

        :return: Iterable around this class.
        """
        self.start()
        self.dataloader_finished = False
        return self

    def __next__(self):
        """
Intended to be used as iterator.

        :return: Next iteration element.
        """
        if self.dataloader_finished is True and self.buffer_queue.empty():
            raise StopIteration()

        return self.buffer_queue.get()

    def __len__(self):
        return len(self.data_loader)

In [15]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast

max_examples = 60_000_000
eval_every_steps = 5000
lr = 4e-5
use_amp = True

model = LanguageModel(
    vocab_size=vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)
train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, num_workers=4)
validation_loader = DataLoader(valid_dataset, batch_size=1024, num_workers=4, )

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, min_lr=1e-5, patience=15000, threshold=1e-1, verbose=True)
scaler=GradScaler()


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        logits = model(input_ids)
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, )
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    with autocast(enabled=use_amp):
        logits = model(input_ids)
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
        loss = nn.functional.cross_entropy(logits, target_ids,)
    return loss.item()

best_validation_ppl = 9999
train_losses = []
n_examples = 0
step = 0
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for train_input_ids, train_target_ids in DataManager(train_loader, device=device, buffer_size=1, data_type=None):
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device))
        train_losses.append(loss)

        # LR scheduler
        scheduler.step(loss)

        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in DataManager(validation_loader, device=device, buffer_size=1, data_type=None)]))
                # Checkpoint to best models found.
                if best_validation_ppl > valid_ppl:
                    # Update the new best perplexity.
                    best_validation_ppl = valid_ppl
                    model.eval()
                    torch.save(model, "best_model.pth")

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        pbar.update(len(train_input_ids))
        if n_examples >= max_examples:
            break

pbar.close()

# Restore best model (checkpoint) found
model = torch.load("best_model.pth")

  0%|          | 5120/60000000 [00:10<122:53:39, 135.61it/s]

0 steps; 0 examples so far; train ppl: 2980.96, valid ppl: 2986.13


  9%|▊         | 5126582/60000000 [02:31<7:28:05, 2041.02it/s] 

5000 steps; 5119414 examples so far; train ppl: 252.19, valid ppl: 188.18


 17%|█▋        | 10247020/60000000 [04:57<10:24:53, 1326.99it/s]

...

 85%|████████▌ | 51198674/60000000 [24:21<1:28:46, 1652.41it/s]

50000 steps; 51193554 examples so far; train ppl: 107.11, valid ppl: 114.26


 94%|█████████▍| 56316478/60000000 [26:54<1:34:27, 649.98it/s] 

55000 steps; 56312382 examples so far; train ppl: 105.98, valid ppl: 113.59


60000830it [28:27, 35139.69it/s]                              


Avaliação no dataset de Teste

In [16]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in tqdm(test_loader)
    ]))

print(f'test perplexity: {test_ppl}')

100%|██████████| 82357/82357 [03:37<00:00, 379.14it/s]

test perplexity: 112.68016745231823


Testando gerar uma sentença

In [17]:
prompt = 'Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to '
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)
    input_ids_truncated = input_ids[-context_size:]# Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    predicted_word = tokenizer.decode([predicted_id,])
    prompt += predicted_word[0]
    print(prompt)

RuntimeError: mat1 dim 1 must match mat2 dim 0